In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf

import vgg16_trainable as vgg16
import os
import time
import numpy as np
import utils
from IPython.display import clear_output

PRETRAINED_MODEL_PATH= None
N_EPOCHS = 300
INIT_LEARNING_RATE = 0.001
WEIGHT_DECAY_RATE = 0.0005
MOMENTUM = 0.9
IMAGE_HEIGHT  = 224    #960
IMAGE_WIDTH   = 224    #720
NUM_CHANNELS  = 3
BATCH_SIZE = 60
N_CLASSES = 2
DROPOUT = 0.50
NUM_GPUS =3
ckpt_dir = "/home/kami/ckpt_dir"
LOGS_PATH = '/home/kami/tensorflow_logs'
WEIGHT_PATH = '.npy'
TRAINSET_PATH = '/home/kami/PycharmProjects/csvgeneration/imagenetdata100-1.csv'
VALSET_PATH ='/home/kami/PycharmProjects/csvgeneration/imagenetdata100-1.csv'

csv_path = tf.train.string_input_producer([TRAINSET_PATH], shuffle=True)
textReader = tf.TextLineReader()
_,csv_content = textReader.read(csv_path)
im_name, im_label = tf.decode_csv(csv_content, record_defaults=[[""], [1]])

im_content = tf.read_file(im_name)
train_image = tf.image.decode_jpeg(im_content, channels=3)
train_image = tf.cast(train_image, tf.float32)/255. # necessary for mapping rgb channels from 0-255 to 0-1 float.
# train_image = augment(train_image)
size = tf.cast([IMAGE_HEIGHT, IMAGE_WIDTH], tf.int32)
train_image = tf.image.resize_images(train_image, size)
train_label = tf.cast(im_label, tf.int64) # unnecessary
train_image_batch, train_label_batch = tf.train.shuffle_batch([train_image, train_label], batch_size=BATCH_SIZE,
                                                               capacity = 1000 + 3*BATCH_SIZE, min_after_dequeue = 1000)

images_splits = tf.split(0, NUM_GPUS, train_image_batch)
labels_splits = tf.split(0, NUM_GPUS, train_label_batch)
        
   

with tf.device('/cpu:0'):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # For populating queues with batches, very important!
    threads = tf.train.start_queue_runners(sess=sess)
    
    train_imbatch, train_labatch = sess.run([images_splits[0], labels_splits[0]])
    print(len(train_imbatch))
    print(len(train_imbatch[0]))
    print(len(train_imbatch[0][0]))
    print(len(train_imbatch[0][0][0]))
    train_imbatch, train_labatch = sess.run([images_splits[1], labels_splits[1]])
    print(len(train_imbatch))
    print(len(train_imbatch[0]))
    print(len(train_imbatch[0][0]))
    print(len(train_imbatch[0][0][0]))
    


20
224
224
3
20
224
224
3
